# PDE on a metric graph solver
by Fabian Visser
2/7/20
https://towardsdatascience.com/how-to-create-animated-graphs-in-python-bb619cc2dec1


In [1]:
%matplotlib notebook

import numpy as np
import scipy as sc
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


## To Edit

In [2]:
sources = np.array([1, 2, 1, 4, 1, 6, 1, 8, 2, 3, 4, 5, 6, 7]) #Starting points of edges
targets = np.array([2, 3, 4, 5, 6, 7, 8, 9, 4, 5, 6, 7, 8, 9]) #Endpoints of edges, in this case we have 4 vertices, and 3 edges
#Length of each edge
L1=3
L2=3
L3=5
L4=5
L5=5
L6=5
L7=3
L8=3
L9=2
L10=4
L11=np.sqrt(2)
L12=2*np.sqrt(2)
L13=2
L14=4
#No. of discretization points of each edge
N1=20
N2=20
N3=20
N4=20
N5=20
N6=20
N7=20
N8=20
N9=20
N10=20
N11=20
N12=20
N13=20
N14=20
#Robin coefficients of each vertex
robinCoeff= [0, 0, 0, 0, 0, 0, 0, 0, 0]
#Length in time
T = 50
#Discretization of time
nt = 1000
#Initial conditions, unique on each edge
f = lambda x : [np.zeros(len(x)), 5*np.exp(-10*(x-1.5)**2), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), 5*np.exp(-10*(x-1.5)**2), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x))]
g = lambda x : [np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x))]


L = np.array([L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12, L13, L14])
N = np.array([N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, N11, N12, N13, N14])


# Run the rest

## Graph Class

In [3]:
class Graph:
    
    #Define Graph.Nodes and Graph.Edges
    def __init__(self, source, target, L, N, rc):
        self.Nodes = self.Nodes(source, target, rc) 
        self.Edges = self.Edges(source, target, L, N, rc)
      

    
    class Nodes:
        def __init__(self, source, target, rc):    
            self.Vertices = sorted(list(set(source) | set(target))) #Collection of vertices of Graph
            self.robinCoeff = rc #Robin coefficients of each Graph
            self.junctIndic = np.nan_to_num(rc,nan=-1)+1 #Indicator function checking if the vertex is a junction
            self.junctNo = np.count_nonzero(self.junctIndic) #Number of junction vertices
            self.x1 = [] #x-position of vertices
            self.x2 = [] #y-position of vertices  
            self.y = [None] * len(self.Vertices) #z-position of vertices (the solution)


    
    
    class Edges:
        def __init__(self, source, target, L, N, rc):    
            self.EndNodes = np.column_stack((source, target)) #Collection of starting and endpoints of edges of Graph
            self.L = L #Length of Edge
            self.nx = [None] * len(N) #Number of discretization points of each edge
            self.dx = [None] * len(L) #Dx of each edge
            self.x = [None] * len(L) #Discretized points
            self.xn = [None] * len(L)
            self.x1 = [] #x-position of each point on each edge
            self.x2 = [] #y-position of each point on each edge
            rc = np.nan_to_num(rc,nan=-1)+1 #Robin coefficients
            for i in range(0,len(L)):
                LNode = int(rc[self.EndNodes[i][0]-1]) 
                RNode = int(rc[self.EndNodes[i][1]-1])
                excpoints = LNode+RNode
                self.nx[i] = N[i]-excpoints
                self.dx[i] = L[i]/(N[i]-1)
                self.x[i] = np.linspace(self.dx[i]*LNode, L[i]-self.dx[i]*RNode, self.nx[i]) #discretized points excluding junctions
                self.xn[i] = np.linspace(self.dx[i], L[i]-self.dx[i], N[i]-2) #discretized points excluding edgepoints
            
            
            
            
    #Create the graph in 3d space
    
    def addjunctCoords(self, L):
        
        #Place the Nodes on the x1,x2 plane
        self.Nodes.x1 = np.array([0, 3, 6, 3, 6, 2, 4, 0, 0])
        self.Nodes.x2 = np.array([0, 0, 0, 2, 4, 3, 6, 3, 6])
        
        xn = self.Edges.xn
        
        #Place the Edges on the x1,x2 plane
        L1X1 = xn[0]
        L2X1 = xn[1]+3
        L3X1 = xn[2]*3/5
        L4X1 = xn[3]*3/5+3
        L5X1 = xn[4]*2/5
        L6X1 = xn[5]*2/5+2
        L7X1 = np.zeros(np.size(xn[6]))
        L8X1 = np.zeros(np.size(xn[7]))
        L9X1 = np.zeros(np.size(xn[8]))+3
        L10X1 = np.zeros(np.size(xn[9]))+6
        L11X1 = 3-xn[10]/np.sqrt(2)
        L12X1 = 6-xn[11]/np.sqrt(2)
        L13X1 = 2-xn[12]
        L14X1 = 4-xn[13]

        L1X2 = np.zeros(np.size(xn[0]))
        L2X2 = np.zeros(np.size(xn[1]))
        L3X2 = xn[2]*2/5
        L4X2 = xn[3]*2/5+2
        L5X2 = xn[4]*3/5
        L6X2 = xn[5]*3/5+3
        L7X2 = xn[6]
        L8X2 = xn[7]+3
        L9X2 = xn[8]
        L10X2 = xn[9]
        L11X2 = xn[10]/np.sqrt(2)+2
        L12X2 = xn[11]/np.sqrt(2)+4
        L13X2 = np.zeros(np.size(xn[12]))+3
        L14X2 = np.zeros(np.size(xn[13]))+6
        

        self.Edges.x1 = np.array([L1X1, L2X1, L3X1, L4X1, L5X1, L6X1, L7X1, L8X1, L9X1, L10X1, L11X1, L12X1, L13X1, L14X1])
        self.Edges.x2 = np.array([L1X2, L2X2, L3X2, L4X2, L5X2, L6X2, L7X2, L8X2, L9X2, L10X2, L11X2, L12X2, L13X2, L14X2]) 

## Functions

In [4]:

#Creates Laplacian of Graph
#Applies central differences on edges, and forward on junctions
def GraphLaplacian(G):
    nx = G.Edges.nx
    nxC = np.insert(np.cumsum(nx),0,[0])
    nxTot = nxC[-1]+G.Nodes.junctNo #Total number of discretized points of graph
    dx=G.Edges.dx;
    M = np.zeros([nxTot,nxTot])
    p = 0
    #Central difference
    for k in range(0,len(G.Edges.EndNodes)):
        n = nx[k]
        e = np.ones(n)
        A = (np.diag(np.delete(e,0),-1)+np.diag(-2*e)+np.diag(np.delete(e,0),1))/dx[k]**2
        M[nxC[k]:nxC[k+1],nxC[k]:nxC[k+1]]=A;
    
    #Forward Difference
    for j in range(1,len(G.Nodes.Vertices)+1):
        if ~np.isnan(G.Nodes.robinCoeff[j-1]):
            degreeHere = len(np.where(G.Edges.EndNodes == j)[0]) #Degree of Node
            allEdges = np.where(G.Edges.EndNodes == j)[0]+1 #Which edges are present at Node
            inOrOut = np.where(G.Edges.EndNodes == j)[1] #Whether the edges enter or leave Node
            for neighbor in range(0,degreeHere): #Looping over each edge
                colBranch = allEdges[neighbor]
                colDirection = inOrOut[neighbor]
                if colDirection == 1: #edge is entering the node
                    col1 = nxC[colBranch]
                    col2 = col1-1
                if colDirection == 0: #edge is leaving the node
                    col1 = nxC[colBranch-1]+1
                    col2 = col1+1
                M[nxC[-1]+p,col1-1] = M[nxC[-1]+p,col1-1] + 1 / (G.Edges.dx[colBranch-1]**2)
                M[col1-1,nxC[-1]+p] = 1 / G.Edges.dx[colBranch-1]**2
            M[nxC[-1]+p,nxC[-1]+p]= sum([(1 / (G.Edges.dx[i-1]**2) - 2 / (G.Edges.dx[i-1]**2)) for i in allEdges])
            p = p+1
    return(M)

#Calculate solution using Forward Euler
def calcSol(G,M,t,f,g):
    Dt = t[1]
    nxC = np.insert(np.cumsum(G.Edges.nx),0,[0])
    nxTot = nxC[-1]+G.Nodes.junctNo
    r = np.zeros(nxTot)
    u = []
    v = []
    p=0 #Counter for junctions
    for h in range(0,len(G.Edges.L)):
        u = np.hstack((u,f(G.Edges.x[h])[h]))
        v = np.hstack((v,g(G.Edges.x[h])[h]))

    #Form f and g into a list where the junctions count once
    for j in range(1,len(G.Nodes.Vertices)+1):
        if ~np.isnan(G.Nodes.robinCoeff[j-1]): #Looking at junctions specifically
            degreeHere = len(np.where(G.Edges.EndNodes == j)[0]) #Degree of Node
            allEdges = np.where(G.Edges.EndNodes == j)[0][0] #Which edges are present at the node
            inOrOut = np.where(G.Edges.EndNodes == j)[1][0] #Whether the edges enter or leave the node
            if inOrOut == 1: #edge is entering the node
                x = np.array([G.Edges.x[allEdges][-1]+G.Edges.dx[allEdges]])
                fjunct = f(x)[allEdges]
                gjunct = g(x)[allEdges]
            if inOrOut == 0: #edge is leaving the node
                fjunct = f(np.array([0]))[allEdges]
                gjunct = g(np.array([0]))[allEdges]
            u = np.hstack((u,fjunct))
            v = np.hstack((v,gjunct))
                    
        else: #Looking at dirichlet boundary points
            allEdges = np.where(G.Edges.EndNodes == j)[0][0]
            inOrOut = np.where(G.Edges.EndNodes == j)[1][0]
            if inOrOut == 1:
                r[nxC[allEdges+1]-1] = f(G.Edges.x[allEdges])[allEdges][-1] / (G.Edges.dx[allEdges]**2)
            if inOrOut == 0:
                r[nxC[allEdges]] = f(G.Edges.x[allEdges])[allEdges][0] / (G.Edges.dx[allEdges]**2)
    W = u
    #Applying Forward Euler
    for i in range(0,len(t)-1):
        u = u + Dt*v
        v = v + Dt*(np.dot(M,u)+r)
        W = np.vstack((W,u)) 
    return W
    

#Calculate x1,x2,y values for line i, such that we can plot it
def PlotGraph(G,W,i):
    nxC = np.insert(np.cumsum(G.Edges.nx),0,[0])
    rc = np.nan_to_num(G.Nodes.robinCoeff,nan=-1)+1
    y = W[nxC[i]:nxC[i+1]]
    x1 = []
    x2 = []
    
    lnode = G.Edges.EndNodes[i][0]
    rnode = G.Edges.EndNodes[i][1]
    LNode = rc[lnode-1]
    RNode = rc[rnode-1]
    if LNode == 1: #If the left node is a junction
        p = np.sum(rc[0:lnode], dtype=int)
        y = np.hstack((W[nxC[-1]+p-1],y))
    if RNode == 1: #If the right node is a junction
        p = np.sum(rc[0:rnode], dtype=int)
        y = np.hstack((y,W[nxC[-1]+p-1]))                            
    x1l = G.Nodes.x1[lnode-1]
    x1r = G.Nodes.x1[rnode-1]
    x1 = np.hstack((x1l, G.Edges.x1[i], x1r))
    x2l = G.Nodes.x2[lnode-1]
    x2r = G.Nodes.x2[rnode-1]
    x2 = np.hstack((x2l, G.Edges.x2[i], x2r))  
    return [x1,x2,y]

## Calculating variables

In [5]:
t = np.linspace(0,T,nt)
G = Graph(sources, targets, L, N, robinCoeff)
G.addjunctCoords(L)

#Calculate Laplacian
M = GraphLaplacian(G)

#Calculate solution over all edges
W = calcSol(G,M,t,f,g)



## Grapher

In [6]:
#Plot the Graph

fig = plt.figure()
ax = fig.gca(projection='3d', xlim = [-1,7], ylim = [-1,7], zlim = [-2,10])
Graphline = [ax.plot([],[],[])[0] for _ in range(0,len(L))]
def init():
    for line in Graphline:
        line.set_data_3d([], [], [])        
    return Graphline

def animate(i): #Looping over time
    for j,line in enumerate(Graphline): #Looping over edges
        [x1,x2,y] = PlotGraph(G, W[i-1], j)
        line.set_data_3d(x1,x2,y)
    return Graphline
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(t), interval=30)

#Create a .mp4 file of the plot

#writervideo = animation.FFMpegWriter(fps=60) 
#anim.save("Spiderweb.mp4")
#HTML(anim.to_html5_video())

<IPython.core.display.Javascript object>

## Laplacian

In [7]:
#Plot Laplacian
df_cm = pd.DataFrame(M)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.show()

<IPython.core.display.Javascript object>